In [28]:
import torch
from torch import Tensor

In [42]:
T = 5
time = torch.linspace(0, 1, T)
print(time, time.shape)
stacked = time.repeat(3, 1)[:,:, None]
print(stacked, stacked.shape)

tensor([0.0000, 0.2500, 0.5000, 0.7500, 1.0000]) torch.Size([5])
tensor([[[0.0000],
         [0.2500],
         [0.5000],
         [0.7500],
         [1.0000]],

        [[0.0000],
         [0.2500],
         [0.5000],
         [0.7500],
         [1.0000]],

        [[0.0000],
         [0.2500],
         [0.5000],
         [0.7500],
         [1.0000]]]) torch.Size([3, 5, 1])
